In [1]:
import imaplib
import email
import smtplib
from email.header import decode_header
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import TypedDict, List, Any, Annotated
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langgraph.prebuilt import ToolNode, tools_condition
import operator
from dotenv import load_dotenv

load_dotenv()

llm_endpoint = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    temperature=0.2,
    max_new_tokens=512,
)

llm = ChatHuggingFace(llm=llm_endpoint)

IMAP_SERVER = "imap.gmail.com"
SMTP_SERVER = "smtp.gmail.com"
EMAIL_ADDRESS = "dominhquang02092004@gmail.com"
APP_PASSWORD = "ndekffuzwhnobldp"

c:\Users\ASUS\Learning-LangGraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def fetch_emails_tool(k: int) -> list:
    """
    Đọc k email mới nhất chưa đọc từ hộp thư đến.
    Args:
        k: Số lượng email tối đa muốn đọc.
    """
    emails_found = []
    try:
        imap = imaplib.IMAP4_SSL(IMAP_SERVER, 993)
        imap.login(EMAIL_ADDRESS, APP_PASSWORD)
        imap.select("INBOX")
        _, messages = imap.search(None, "UNSEEN")
        
        if messages[0]:
            ids = messages[0].split()[-k:][::-1]
            for mail_id in ids:
                _, msg_data = imap.fetch(mail_id, "(RFC822)")
                msg = email.message_from_bytes(msg_data[0][1])
                
                subject, encoding = decode_header(msg.get("Subject", ""))[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding or "utf-8")
                
                emails_found.append({
                    "from": msg.get("From"),
                    "subject": subject,
                    "id": mail_id.decode()
                })
        imap.logout()
    except Exception as e:
        return [f"Lỗi khi đọc mail: {str(e)}"]
    return emails_found

def send_email_tool(to_email: str, subject: str, body: str) -> str:
    """
    Gửi email đến một địa chỉ cụ thể.
    Args:
        to_email: Địa chỉ email người nhận.
        subject: Tiêu đề thư.
        body: Nội dung thư.
    """
    try:
        msg = MIMEMultipart()
        msg["From"] = EMAIL_ADDRESS
        msg["To"] = to_email
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain", "utf-8"))
        
        with smtplib.SMTP(SMTP_SERVER, 587) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, APP_PASSWORD)
            server.sendmail(EMAIL_ADDRESS, [to_email], msg.as_string())
        return f"✅ Gửi thành công tới {to_email}"
    except Exception as e:
        return f"❌ Lỗi gửi mail: {str(e)}"

In [3]:
tools = [fetch_emails_tool, send_email_tool]


model = ChatHuggingFace(llm=llm_endpoint).bind_tools(tools)

tool_node = ToolNode(tools)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[List[HumanMessage], operator.add]

In [5]:
def agent_node(state: AgentState):
    """Gọi LLM để quyết định hành động tiếp theo."""
    response = model.invoke(state["messages"])
    return {"messages": [response]}

builder = StateGraph(AgentState)

builder.add_node("email_agent", agent_node)
builder.add_node("execute_tools", ToolNode(tools))

builder.set_entry_point("email_agent")

builder.add_conditional_edges(
    "email_agent",
    tools_condition, 
    {
        "tools": "execute_tools", 
        "__end__": END            
    }
)

builder.add_edge("execute_tools", "email_agent")

graph = builder.compile()

In [7]:
graph.invoke({"messages": [HumanMessage(content="giúp tôi soạn một email có nội dung như sau: Xin chào, tôi muốn đặt một cuộc hẹn vào ngày mai. Địa chỉ email target là dominhquang_t67@hus.edu.vn") ]})

{'messages': [HumanMessage(content='giúp tôi soạn một email có nội dung như sau: Xin chào, tôi muốn đặt một cuộc hẹn vào ngày mai. Địa chỉ email target là dominhquang_t67@hus.edu.vn', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"to_email":"dominhquang_t67@hus.edu.vn","subject":"Yêu cầu đặt cuộc hẹn","body":"Xin chào,\\nTôi muốn đặt một cuộc hẹn vào ngày mai.\\n\\nTrân trọng,\\n[Your Name]"}', 'name': 'send_email_tool', 'description': None}, 'id': 'call_kgs1im682hqypko3d0rz8k83', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 409, 'total_tokens': 483}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bf5e3-c003-7480-b6a0-1117378c155e-0', tool_calls=[{'name': 'send_email_tool', 'args': {'to_email': 'dominhquang_t67@hus.edu.vn', 'subject': 'Yêu cầu đặt cuộc hẹn', 'b